<h1>Grafo delle co-authorship relativo alle 3 discipline</h1>
<p>Il grafo mostra:
<ul>
    <li>tutti i professori di informatica coinvolti</li>
    <li>i legami di co-authorship dalle tre discipline</li>
    <li>i nodi colorati in base alle aree in cui pubblicano</li>
    <li>i ponti interdisciplinari (nodi viola o neri)</li>
    <li>il peso degli archi (numero di articoli)</li>
</ul>
</p>

In [47]:
from pyvis.network import Network
import networkx as nx
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter

<h2>Caricare i pickle dei grafi delle singole discipline</h2>

In [48]:
with open("./grafi_pickle/G_inf.pkl", "rb") as f:
    G_inf = pickle.load(f)

with open("./grafi_pickle/G_mat.pkl", "rb") as f:
    G_mat = pickle.load(f)

with open("./grafi_pickle/G_fis.pkl", "rb") as f:
    G_fis = pickle.load(f)

<h2>Creazione del grafo</h2>

In [49]:
# grafo vuoto che conterrà tutto
G_informatica_matematica_fisica = nx.Graph()

<h3>Funzioni utili per la creazione del grafo</h3>

In [50]:

# Funzione per assicurare l'attributo 'disciplines' sul nodo
def ensure_node_disciplines(node):
    if "disciplines" not in G_informatica_matematica_fisica.nodes[node]:
        # crea l'attributo come set (se non esiste)
        G_informatica_matematica_fisica.nodes[node]["disciplines"] = set()

# Funzione che aggiunge un grafo di origine al grafo delle co-authorship delle tre discipline
def aggiungi_grafo(G_orig, disciplina):
    # 1) aggiungi archi (mantengo somma dei pesi ed elenco discipline per arco)
    for u, v, data in G_orig.edges(data=True):
        weight = data.get("weight", 1)

        if G_informatica_matematica_fisica.has_edge(u, v):
            # sommare i pesi
            if "weight" in G_informatica_matematica_fisica[u][v]:
                G_informatica_matematica_fisica[u][v]["weight"] += weight
            else:
                G_informatica_matematica_fisica[u][v]["weight"] = weight
            # aggiungi disciplina all'insieme (crealo se manca)
            if "disciplines" in G_informatica_matematica_fisica[u][v]:
                G_informatica_matematica_fisica[u][v]["disciplines"].add(disciplina)
            else:
                G_informatica_matematica_fisica[u][v]["disciplines"] = {disciplina}
        else:
            # add_edge aggiunge anche i nodi se mancanti
            G_informatica_matematica_fisica.add_edge(u, v, weight=weight, disciplines={disciplina})

    # 2) assicurati che tutti i nodi del grafo originale esistano nel grafo delle co-authorship delle tre discipline
    for n in G_orig.nodes():
        if n not in G_informatica_matematica_fisica.nodes:
            # aggiungi il nodo con l'attributo disciplines
            G_informatica_matematica_fisica.add_node(n, disciplines={disciplina})
        else:
            # se esiste ma non ha 'disciplines', crealo; poi aggiungi la disciplina
            if "disciplines" not in G_informatica_matematica_fisica.nodes[n]:
                G_informatica_matematica_fisica.nodes[n]["disciplines"] = set()
            G_informatica_matematica_fisica.nodes[n]["disciplines"].add(disciplina)


In [51]:
# Unisci i tre grafi
aggiungi_grafo(G_inf, "informatica")
aggiungi_grafo(G_mat, "matematica")
aggiungi_grafo(G_fis, "fisica")

with open("./grafi_pickle/G_unificato.pkl", "wb") as f:
    pickle.dump(G_informatica_matematica_fisica, f)

# --- 3) Controllo rapido ---
print("Nodi nel grafo delle co-authorship delle tre discipline:", G_informatica_matematica_fisica.number_of_nodes())
print("Archi nel grafo delle co-authorship delle tre discipline:", G_informatica_matematica_fisica.number_of_edges())

Nodi nel grafo delle co-authorship delle tre discipline: 36
Archi nel grafo delle co-authorship delle tre discipline: 66


<h2>Visualizzazione del grafo tramite libreria Pyvis</h2>
<p>Il grafo è salvato in un file .html nella relativa cartella</p>

In [52]:
net = Network(height="800px", width="100%",notebook=True,cdn_resources="remote",filter_menu=True, bgcolor="black", font_color="white")

<h3>Funzione per colorare i nodi</h3>
<p>I nodi del grafo sono colorati diversamente in base al contributo dei professori nelle varie discipline</p>

In [53]:
def colore_nodo(disc):
    if len(disc) == 1:
        if "informatica" in disc:
            return "#4A90E2"   # blu
        if "matematica" in disc:
            return "#2ECC71"   # verde
        if "fisica" in disc:
            return "#F5A623"   # arancione
    elif len(disc) == 2:
        return "#9B59B6"  # viola → interdisciplinare
    else:
        return "white"  # white → autore che pubblica in tutte e tre

def shape_nodo(disc):
    if len(disc) == 1:
        return "dot"
    elif len(disc) == 2:
        return "triangle"
    else:
        return "star"

In [54]:
# aggiungi nodi
for node, data in G_informatica_matematica_fisica.nodes(data=True):
    disc = data["disciplines"]
    net.add_node(
        node,
        label=node,
        color=colore_nodo(disc),
        title=" | ".join(disc),
        size=12 + 2*len(disc),
        shape=shape_nodo(disc),
        font_size=12
    )

# aggiungi archi
for u, v, data in G_informatica_matematica_fisica.edges(data=True):
    net.add_edge(
        u,
        v,
        value=data["weight"],
        title=f"Weight = {data['weight']} | {', '.join(data['disciplines'])}"
    )

# salva html
net.write_html("../html/grafi_co-authorship/grafo_unificato.html")
print("Creato: grafo_co-authorship_unificato.html")
net.show("../html/grafi_co-authorship/grafo_unificato.html")

Creato: grafo_co-authorship_unificato.html
../html/grafi_co-authorship/grafo_unificato.html
